In [ ]:
!pip install tabulate

In [1]:
import numpy as np
from tabulate import tabulate
class FuzzySet:
    def __init__(self, iterable: any):
        self.f_set = set(iterable)
        self.f_list = list(iterable)
        self.f_len = len(iterable)
        for elem in self.f_set:
            if not isinstance(elem, tuple):
                raise TypeError("No tuples in the fuzzy set")
            if not isinstance(elem[1], float):
                raise ValueError("Probabilities not assigned to elements")

    def __or__(self, other):
        # fuzzy set union
        if len(self.f_set) != len(other.f_set):
            raise ValueError("Length of the sets is different")
        f_set = [x for x in self.f_set]
        other = [x for x in other.f_set]
        return FuzzySet([f_set[i] if f_set[i][1] > other[i][1] else other[i] for i in range(len(self))])

    def __and__(self, other):
        # fuzzy set intersection
        if len(self.f_set) != len(other.f_set):
            raise ValueError("Length of the sets is different")
        f_set = [x for x in self.f_set]
        other = [x for x in other.f_set]

        return FuzzySet([f_set[i] if f_set[i][1] < other[i][1] else other[i] for i in range(len(self))])

    def __invert__(self):
        f_set = [x for x in self.f_set]
        for indx, elem in enumerate(f_set):
            f_set[indx] = (elem[0], float(round(1 - elem[1], 2)))
        return FuzzySet(f_set)

    def __sub__(self, other):
        if len(self) != len(other):
            raise ValueError("Length of the sets is different")
        return self & ~other

    def __mul__(self, other):
        if len(self) != len(other):
            raise ValueError("Length of the sets is different")
        return FuzzySet([(self[i][0], self[i][1] * other[i][1]) for i in range(len(self))])

    def __mod__(self, other):
        # cartesian product
        print(f'The size of the relation will be: {len(self)}x{len(other)} ')
        mx = self
        mi = other
        tmp = [[] for i in range(len(mx))]
        i = 0
        for x in mx:
            for y in mi:
                tmp[i].append(min(x[1], y[1]))
            i += 1
        return np.array(tmp)

    @staticmethod
    def max_min(array1: np.ndarray, array2: np.ndarray):
        tmp = np.zeros((array1.shape[0], array2.shape[1]))
        t = list()
        for i in range(len(array1)):
            for j in range(len(array2[0])):
                for k in range(len(array2)):
                    t.append(round(min(array1[i][k], array2[k][j]), 2))
                tmp[i][j] = max(t)
                t.clear()
        return tmp

    def __len__(self):
        self.f_len = sum([1 for i in self.f_set])
        return self.f_len

    def __str__(self):
        return f'{[x for x in self.f_set]}'

    def __getitem__(self, item):
        return self.f_list[item]

    def __iter__(self):
        for i in range(len(self)):
            yield self[i]


a = FuzzySet({('x1', 0.5), ('x2', 0.2), ('x3', 0.0)})
b = FuzzySet({('x1', 0.8), ('x2', 0.7), ('x3', 1.0)})
c = FuzzySet({('x', 0.3), ('y', 0.3), ('z', 0.5)})
x = FuzzySet({('a', 0.5), ('b', 0.3), ('c', 0.7)})
y = FuzzySet({('a', 0.6), ('b', 0.4)})

# Displaying the results in a tabular format
headers = ["Operation", "Result"]
data = [
    ["a", a],
    ["b", b],
    ["Fuzzy union", a | b],
    ["Fuzzy intersection", a & b],
    ["Fuzzy inversion of b", ~b],
    ["Fuzzy inversion of a", ~a],
    ["Fuzzy Subtraction", a - b],
]

# Print the table using tabulate
print(tabulate(data, headers=headers, tablefmt="fancy_grid"))

r = np.array([[0.6, 0.6, 0.8, 0.9], [0.1, 0.2, 0.9, 0.8], [0.9, 0.3, 0.4, 0.8], [0.9, 0.8, 0.1, 0.2]])
s = np.array([[0.1, 0.2, 0.7, 0.9], [1.0, 1.0, 0.4, 0.6], [0.0, 0.0, 0.5, 0.9], [0.9, 1.0, 0.8, 0.2]])

# Display the Max Min results in a tabular format
max_min_result = FuzzySet.max_min(r, s)
max_min_headers = [f"Column {i+1}" for i in range(max_min_result.shape[1])]
max_min_table = tabulate(max_min_result, headers=max_min_headers, tablefmt="fancy_grid")

print(f"\nMax Min:\n{max_min_table}")

╒══════════════════════╤═════════════════════════════════════════╕
│ Operation            │ Result                                  │
╞══════════════════════╪═════════════════════════════════════════╡
│ a                    │ [('x2', 0.2), ('x1', 0.5), ('x3', 0.0)] │
├──────────────────────┼─────────────────────────────────────────┤
│ b                    │ [('x2', 0.7), ('x3', 1.0), ('x1', 0.8)] │
├──────────────────────┼─────────────────────────────────────────┤
│ Fuzzy union          │ [('x2', 0.7), ('x3', 1.0), ('x1', 0.8)] │
├──────────────────────┼─────────────────────────────────────────┤
│ Fuzzy intersection   │ [('x2', 0.2), ('x1', 0.5), ('x3', 0.0)] │
├──────────────────────┼─────────────────────────────────────────┤
│ Fuzzy inversion of b │ [('x1', 0.2), ('x3', 0.0), ('x2', 0.3)] │
├──────────────────────┼─────────────────────────────────────────┤
│ Fuzzy inversion of a │ [('x3', 1.0), ('x2', 0.8), ('x1', 0.5)] │
├──────────────────────┼──────────────────────────────────────